In [ ]:
%%capture
!pip install transformers
!pip install torch
!pip install transformers[torch]
!pip install transformers --upgrade
!pip install datasets
!pip install transformers[sentence-transformers]
!pip install tinybert

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"nhqanh","key":"67cf9f45d1a5c201386c417709e95663"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# https://technowhisp.com/kaggle-api-python-documentation/
# Authenticate to Kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Download the dataset
api.dataset_download_files('duyminhnguyentran/csc15105', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/duyminhnguyentran/csc15105


**Tokenizer** - Convert the input to fit the model

In [ ]:
# from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, Trainer, TrainingArguments
import json
import torch
import numpy as np
from datasets import load_metric

from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments

# Load TinyBERT model and tokenizer
model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


# Load and preprocess the data
with open('Project2_Data/train.json', 'r') as f:
    df = json.load(f)
with open('Project2_Data/train.json', 'r') as f:
    test_data = json.load(f)

# Prepare the dataset
def prepare_dataset(data):
    context = data['context']
    qas = data['qas']

    prepared_samples = []

    for qa in qas:
        input_text = f"{context} {qa['question']}"
        answer_text = qa['answers'][0]['text']

        inputs = tokenizer(input_text, padding='max_length', truncation=True, return_tensors='pt', max_length=512)

        # Find start and end position of answer in context
        start_idx = context.find(answer_text)
        end_idx = start_idx + len(answer_text) - 1

        # Prepare the sample
        prepared_sample = {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'start_positions': torch.tensor([start_idx]),
            'end_positions': torch.tensor([end_idx]),
        }

        prepared_samples.append(prepared_sample)

    return prepared_samples

train_dataset = [sample for data in df for sample in prepare_dataset(data)]
eval_dataset = [sample for data in test_data for sample in prepare_dataset(data)]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Fine-tuning the Model based on Project Dataset**

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./output',
    learning_rate=1e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=3,
    fp16=True,
    evaluation_strategy="epoch"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Fine-tune the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,6.105500,6.088961
2,6.071400,6.047335
3,6.043600,6.012599
4,6.015200,5.981456
5,5.988700,5.957073
6,5.977400,5.955251
7,5.960600,5.919493
8,5.940400,5.910410
9,5.935800,5.913580
10,5.935100,5.906439


TrainOutput(global_step=15640, training_loss=6.002567823951506, metrics={'train_runtime': 5044.5526, 'train_samples_per_second': 99.208, 'train_steps_per_second': 3.1, 'total_flos': 7025956434616320.0, 'train_loss': 6.002567823951506, 'epoch': 10.0})

**Evaluate the trained model**

In [ ]:
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)


Evaluation Results: {'eval_loss': 5.906439304351807, 'eval_runtime': 122.7237, 'eval_samples_per_second': 407.794, 'eval_steps_per_second': 25.488, 'epoch': 10.0}


In [ ]:
# Load the metric for question answering
squad_metric = load_metric("squad")

# Evaluate the model
eval_predictions = trainer.predict(eval_dataset)

<ipython-input-7-e0049c5981ec>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# Post-process the predictions
predictions = []
references = []

for i, sample in enumerate(eval_dataset):
    start_logits, end_logits = torch.from_numpy(eval_predictions.predictions[0][i]), torch.from_numpy(eval_predictions.predictions[1][i])
    start_pred = torch.argmax(start_logits, axis=-1).item()
    end_pred = torch.argmax(end_logits, axis=-1).item()

    # Get the predicted answer span
    pred_answer = tokenizer.decode(sample["input_ids"][start_pred:end_pred+1])

    # Get the reference answer
    reference = tokenizer.decode(sample["input_ids"][sample["start_positions"]:sample["end_positions"]+1])

    # Append to lists
    predictions.append({"id": str(i), "prediction_text": pred_answer})
    references.append({"id": str(i), "answers": {"text": [reference]}})


# Compute the metrics
results = squad_metric.compute(predictions=predictions, references=references)

# Extract EM and F1 score
em = results["exact_match"]
f1 = results["f1"]

print(f"Exact Match (EM): {em}")
print(f"F1 Score: {f1}")


Exact Match (EM): 1.2368620868800704
F1 Score: 2.4112111489325607


**Generate Answer**

MODEL 2: **DISTILLBERT**

**Tokenizer** - Convert the input to fit the model

In [ ]:
import json
import torch
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_metric

# Load DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

# Load and preprocess the data
with open('Project2_Data/train.json', 'r') as f:
    df = json.load(f)
with open('Project2_Data/train.json', 'r') as f:
    test_data = json.load(f)

# Prepare the dataset
def prepare_dataset(data):
    context = data['context']
    qas = data['qas']

    prepared_samples = []

    for qa in qas:
        input_text = f"{context} {qa['question']}"
        answer_text = qa['answers'][0]['text']

        inputs = tokenizer(input_text, padding='max_length', truncation=True, return_tensors='pt', max_length=512)

        # Find start and end position of answer in context
        start_idx = context.find(answer_text)
        end_idx = start_idx + len(answer_text) - 1

        # Prepare the sample
        prepared_sample = {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'start_positions': torch.tensor([start_idx]),
            'end_positions': torch.tensor([end_idx]),
        }

        prepared_samples.append(prepared_sample)

    return prepared_samples

train_dataset = [sample for data in df for sample in prepare_dataset(data)]
eval_dataset = [sample for data in test_data for sample in prepare_dataset(data)]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Fine-tuning the Model based on Project Dataset**

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./output',
    learning_rate=1e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=3,
    fp16=True,
    evaluation_strategy="epoch"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Custom callback function to print training information after each epoch
# def print_train_info_callback(eval_args, epoch, training_stats):
#    print(f"***** Epoch {epoch} *****")
#    print("Training results:")
#    for key, value in training_stats.items():
##        print(f"  {key}: {value}")
  #  print("")

# Attach the custom callback to Trainer
#trainer.add_callback(print_train_info_callback)
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.987800,5.877581
2,5.732700,5.579824
3,5.564200,5.405827
4,5.445800,5.295693
5,5.333200,5.189556


**Evaluate the trained model**

In [ ]:
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)


In [ ]:
# Define evaluation metric (assuming you care about exact match)
metric = load_metric('squad')

# Evaluate the model on the test set
predictions = trainer.predict(eval_dataset)

# Extract predictions and labels
predictions_list = predictions.predictions[0].tolist()  # Access the first element of the tuple
label_ids_list = predictions.label_ids.tolist()

# Calculate accuracy using the defined metric
accuracy = metric.compute(predictions=predictions_list, references=label_ids_list)

print("Exact Match Accuracy:", accuracy['exact_match'])
